In [1]:
import argparse
import cv2
import numpy as np
import os
import torch
import pdb

from utils import setup_seed, read_points, read_calib, read_label, \
    keep_bbox_from_image_range, keep_bbox_from_lidar_range, vis_pc, \
    vis_img_3d, bbox3d2corners_camera, points_camera2image, \
    bbox_camera2lidar

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def point_range_filter(pts, point_range=[0, -39.68, -3, 69.12, 39.68, 1]):
    '''
    data_dict: dict(pts, gt_bboxes_3d, gt_labels, gt_names, difficulty)
    point_range: [x1, y1, z1, x2, y2, z2]
    논문에 표기된 대로 car, pedstrian cyclist를 검출하는 포인터 범위
    '''
    flag_x_low = pts[:, 0] > point_range[0] # 0
    flag_y_low = pts[:, 1] > point_range[1] # -39.68
    flag_z_low = pts[:, 2] > point_range[2] # -3
    flag_x_high = pts[:, 0] < point_range[3] # 69.12
    flag_y_high = pts[:, 1] < point_range[4] # 39.68
    flag_z_high = pts[:, 2] < point_range[5] # 1
    keep_mask = flag_x_low & flag_y_low & flag_z_low & flag_x_high & flag_y_high & flag_z_high
    pts = pts[keep_mask]
    return pts 

In [39]:
import numpy as np
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
from model.anchors import Anchors, anchor_target, anchors2bboxes
from ops import Voxelization, nms_cuda
from utils import limit_period


class PillarLayer(nn.Module):
    def __init__(self, voxel_size, point_cloud_range, max_num_points, max_voxels):
        super().__init__()
        self.voxel_layer = Voxelization(voxel_size=voxel_size,
                                        point_cloud_range=point_cloud_range,
                                        max_num_points=max_num_points,
                                        max_voxels=max_voxels)

    @torch.no_grad()
    def forward(self, batched_pts):
        '''
        batched_pts: list[tensor], len(batched_pts) = bs
        return: 
               pillars: (p1 + p2 + ... + pb, num_points, c), 
               coors_batch: (p1 + p2 + ... + pb, 1 + 3), 
               num_points_per_pillar: (p1 + p2 + ... + pb, ), (b: batch size)
        '''
        pillars, coors, npoints_per_pillar = [], [], []
        for i, pts in enumerate(batched_pts):
            voxels_out, coors_out, num_points_per_voxel_out = self.voxel_layer(pts) 
            # voxels_out: (max_voxel, num_points, c), coors_out: (max_voxel, 3)
            # num_points_per_voxel_out: (max_voxel, )
            pillars.append(voxels_out)
            coors.append(coors_out.long())
            npoints_per_pillar.append(num_points_per_voxel_out)
        
        pillars = torch.cat(pillars, dim=0) # (p1 + p2 + ... + pb, num_points, c)
        npoints_per_pillar = torch.cat(npoints_per_pillar, dim=0) # (p1 + p2 + ... + pb, )
        coors_batch = []
        for i, cur_coors in enumerate(coors):
            coors_batch.append(F.pad(cur_coors, (1, 0), value=i))
        coors_batch = torch.cat(coors_batch, dim=0) # (p1 + p2 + ... + pb, 1 + 3)

        return pillars, coors_batch, npoints_per_pillar


In [41]:
pc_folder_path = "dataset/testing/velodyne/"
pc_file_list = os.listdir(pc_folder_path)
pc = read_points(pc_folder_path + "/" + pc_file_list[0])

pc = point_range_filter(pc)
# pc_torch = torch.from_numpy(pc)

In [49]:
pc

array([[22.392,  4.376,  0.981,  0.38 ],
       [22.478,  4.466,  0.984,  0.27 ],
       [21.948,  4.432,  0.966,  0.21 ],
       ...,
       [ 3.741, -1.402, -1.747,  0.47 ],
       [ 3.74 , -1.388, -1.744,  0.   ],
       [ 5.584, -1.849, -2.627,  0.   ]], dtype=float32)

In [53]:
a = np.where(pc==[ 4.689,  4.483, -1.64,  0.3])

for i in pc:
    if 4.6 <i[0] < 4.7:
        print(i)

[ 4.665 13.516  0.693  0.5  ]
[ 4.603 -9.284  0.56   0.61 ]
[ 4.643 -9.291  0.56   0.62 ]
[ 4.684 -9.301  0.561  0.63 ]
[4.619 8.013 0.463 0.15 ]
[4.653 9.024 0.488 0.12 ]
[4.653 9.165 0.491 0.27 ]
[ 4.605 -9.256  0.493  0.5  ]
[ 4.633 -9.276  0.494  0.39 ]
[ 4.668 -9.274  0.494  0.5  ]
[4.625 8.153 0.422 0.34 ]
[4.651 8.771 0.434 0.27 ]
[4.605 8.75  0.433 0.43 ]
[ 4.603 -9.296  0.445  0.36 ]
[ 4.639 -9.296  0.445  0.46 ]
[ 4.686 -9.315  0.446  0.38 ]
[4.641 8.143 0.35  0.41 ]
[4.664 8.243 0.352 0.21 ]
[4.638 8.666 0.357 0.46 ]
[ 4.616 -9.235  0.365  0.34 ]
[ 4.658 -9.246  0.365  0.36 ]
[ 4.695 -9.247  0.365  0.36 ]
[4.676 8.119 0.302 0.39 ]
[4.649 8.282 0.303 0.28 ]
[ 4.618 -9.296  0.312  0.58 ]
[ 4.661 -9.308  0.312  0.58 ]
[ 4.699 -9.311  0.313  0.58 ]
[ 4.667 -6.277  0.286  0.25 ]
[ 4.672 -6.243  0.286  0.51 ]
[ 4.677 -6.229  0.286  0.38 ]
[4.666 8.118 0.236 0.31 ]
[4.699 8.572 0.238 0.18 ]
[ 4.633 -9.21   0.239  0.39 ]
[ 4.653 -9.213  0.239  0.32 ]
[ 4.694 -9.223  0.239  0.47 ]
[ 

In [43]:
if __name__ == "__main__":

    np.random.seed(22)

    device = torch.device("cuda:0")

    nclasses=3
    voxel_size=[0.16, 0.16, 0.16]
    point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1]
    max_num_points=32
    max_voxels=(16000, 40000)

    test_layer = PillarLayer(voxel_size=voxel_size,
                            point_cloud_range=point_cloud_range,
                            max_voxels=max_voxels,
                            max_num_points=max_num_points)

    # pc = np.random.uniform(-2, 8, size=[1000, 4]).astype(np.float32)
    pc_tensor = torch.from_numpy(pc).to(device)

    voxels_th, indices_th, num_p_in_vx_th = test_layer([pc_tensor])

    print(voxels_th.size())
    print(indices_th.size())
    print(num_p_in_vx_th.size())

    # voxels_th = voxels_th.cpu().numpy()
    # indices_th = indices_th.cpu().numpy()
    # num_p_in_vx_th = num_p_in_vx_th.cpu().numpy()

    print(voxels_th[0])
    print(indices_th[0])
    print(num_p_in_vx_th[0])



torch.Size([16000, 32, 4])
torch.Size([16000, 4])
torch.Size([16000])
tensor([[22.3920,  4.3760,  0.9810,  0.3800],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 

In [20]:
from spconv.utils import Point2VoxelCPU3d
from spconv.pytorch.utils import PointToVoxel, gather_features_by_pc_voxel_id


In [24]:
coors_batch = []

cur_list = torch.Tensor([[10,10,10],
                        [11,11,11]])

for i, cur_coors in enumerate(cur_list):
    coors_batch.append(F.pad(cur_coors, (1, 0), value=i))

In [25]:
coors_batch = torch.cat(coors_batch, dim=0)

In [26]:
coors_batch = torch.cat(coors_batch, dim=0)

TypeError: cat() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [27]:
coors_batch

tensor([ 0., 10., 10., 10.,  1., 11., 11., 11.])